In [1]:
cd Downloads/

/Users/hasegawatakashikana/Downloads


In [2]:
import pickle
import re
import os

In [3]:
cd movieclassifier/

/Users/hasegawatakashikana/Downloads/movieclassifier


In [4]:
from vectorizer import vect

In [5]:
clf = pickle.load(open(os.path.join("pkl_objects","classifier.pkl"),"rb"))

In [10]:
import numpy as np
label = {0:"negative",1:"positive"}
example =["I love this movie"]
X = vect.transform(example)
print("Prediction: %s\nProbability: %.2f%%" %(label[clf.predict(X)[0]],np.max(clf.predict_proba(X))*100))

Prediction: positive
Probability: 82.48%


In [35]:
import sqlite3
if os.path.exists("reviews.sqlite"):
    os.remove("reviews.sqlite")
conn = sqlite3.connect("reviews.sqlite")
c = conn.cursor()
c.execute("CREATE TABLE review_db (review TEXT, sentiment INTEGER, date TEXT)")
example1 = "I love this movie"
c.execute("INSERT INTO review_db (review,sentiment,date) VALUES (?, ?,DATETIME('now'))",(example1,1))
# c.execute("INSERT INTO review_db (review, sentiment, date) VALUES (?, ?, DATETIME('now'))", (example1, 1))

example2 = "I disliked this movie"
c.execute("INSERT INTO review_db (review,sentiment,date) VALUES (?,?,DATETIME('now'))",(example2,0))
conn.commit()
conn.close()

In [38]:
if os.path.exists('reviews.sqlite'):
    os.remove('reviews.sqlite')

conn = sqlite3.connect('reviews.sqlite')
c = conn.cursor()
c.execute('CREATE TABLE review_db (review TEXT, sentiment INTEGER, date TEXT)')

example1 = 'I love this movie'
c.execute("INSERT INTO review_db (review, sentiment, date) VALUES (?, ?, DATETIME('now'))", (example1, 1))

example2 = 'I disliked this movie'
c.execute("INSERT INTO review_db (review, sentiment, date) VALUES (?, ?, DATETIME('now'))", (example2, 0))

conn.commit()
conn.close()

In [37]:
cd 1st_flask_app_3

/Users/hasegawatakashikana/Downloads/1st_flask_app_3


In [30]:
conn = sqlite3.connect("reviews.sqlite")
c = conn.cursor()
c.execute("SELECT * FROM review_db WHERE date BETWEEN '2018-01-01 00:00:00'AND DATETIME('now')")
results = c.fetchall()
conn.close()
print(results)

[('I love this movie', 1, '2019-05-02 08:51:40'), ('I disliked this movie', 0, '2019-05-02 08:51:40')]


In [31]:
!pip install flask

In [32]:
cd ..

/Users/hasegawatakashikana/Downloads


In [ ]:
from flask import Flask,render_template

app = Flask(__name__)

@app.route('/')
def index():
    return render_template("first_app.html")

if __name__ = "__main__":
    app.run()

In [33]:
!pip install wtforms

     |████████████████████████████████| 174kB 3.6MB/s eta 0:00:01


In [ ]:
from flask import Flask, render_template, request
from wtforms import Form, TextAreaField, validators
import pickle
import sqlite3
import os
import numpy as np

#ローカルディスクからHashingVectorizerをインポート
from vectorizer import vect

app = Flask(__name__)

#分類器の準備
cur_dir = os.path.dirname(__file__)
clf = pickle.load(open(os.path.join(cur_dir,"pkkl_objects","classifier.pkl"),"rb"))

db = os.path.join(cur_dir,"reviews.sqlite")

def classify(document):
    label = {0: "negative",1:"positive"}
    X = vect.transform([document])
    y = clf.predict(X)[0]
    proba = clf.predict_proba(X).max()
    return label[y],proba

def train(document,y):
    X = vect.transform(X)[0]
    clf.partial_fit(X,[y])
    
def sqlite_entry(path,document,y):
    conn = sqlite3.connect(path)
    c = conn.cursor()
    c.execute("INSERT INTO review_db (review,sentiment,date) VALUES (?,?,DATETIME('now'))",(document,y))
    conn.commit()
    conn.close()
    
###Flask
class ReviewForm(Form):
    moviereview = TextAreaField("",[validators.DataRequired(),validators.length(min=15)])
    
@app.route("/")
def index():
    form = ReviewForm(request.form)
    return render_template("reviewform.html",form=form)
@app.route("/results",methods=["POST"])
def results():
    form = ReviewForm(request.form)
    if request.method == "POST" and form.validate():
        review = request.form["moviereview"]
        y,proba = classify(review)
        return render_template("results.html",content=review,prediction=y,probability=round(proba*100,2))
    return render_template("reviewform.html",form=form)

@app.route("/thanks",methods=["POST"])
def feedback():
    feedback = request.form["feedback_button"]
    review = request.form["review"]
    prediction = request.form["prediction"]
    
    inv_label = {0:"negative",1:"positive"}
    y = inv_label[prediction]
    if feedback == "Incorrect":
        y = int(not(y))
    train(review,y)
    sqlite_entry(db,review,y)
    return render_template("thanks.html")

if __name__ == "__main__":
    app.run(debug =True)